In [ ]:
import json
import requests
import os

from pycocotools.coco import COCO
import numpy as np
import cv2

In [ ]:
with open("selected_images.json", "r") as fp: 
    selected_images = json.load(fp)

In [ ]:
# Download selected images
folder_original = "assets/original"
os.makedirs(folder_original, exist_ok=True)
for i in selected_images['images']:
    downloaded_image = requests.get(i['coco_url'])
    with open(os.path.join(folder_original, i['file_name']), 'wb') as fp:
        fp.write(downloaded_image.content)
    # Large version
    image = np.asarray(bytearray(downloaded_image.content), dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    large_image = cv2.resize(image, dsize=None, fx = 10, fy = 10, interpolation = cv2.INTER_AREA)
    filename = os.path.splitext(i['file_name'])
    cv2.imwrite(os.path.join(folder_original, f"{filename[0]}_10xlarge{filename[1]}"), large_image)

In [ ]:
# Get image ids
selected_image_ids = [i['id'] for i in selected_images['images']]

In [ ]:
# Get annotations for given image id

cc = COCO(annotation_file="selected_images.json")

for id_ in selected_image_ids:
    annotations = [a for a in selected_images['annotations'] if a['image_id'] == id_ and a['category_id'] == 1]
    masks = np.array([cc.annToMask(a) for a in annotations])
    mask = masks.max(0).astype(np.uint8)*255
    cv2.imwrite(os.path.join(folder_original, str(id_).zfill(12) +  "_mask.png"), mask)
    # Large version
    large_mask = cv2.resize(mask, dsize=None, fx = 10, fy = 10, interpolation = cv2.INTER_AREA)
    cv2.imwrite(os.path.join(folder_original, str(id_).zfill(12) +  "_10xlarge_mask.png"), large_mask)